<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающие-деревья" data-toc-modified-id="Решающие-деревья-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Решающие деревья</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающие-деревья" data-toc-modified-id="Решающие-деревья-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Решающие деревья</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Отток клиентов

## Постановка задачи
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
data = pd.read_csv('C:/Users/Alex/proj/portfolio/beta_bank/Churn.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


В данных содержатся столбцы, не несущие полезной информации об уходе человека RowNumber,CustomerId,Surname, от них можно избавиться.        
В стобце Tenure есть пропуски. Заменим на 0, так как NaN скорее всего означает, что у клиента ничего нет во владении.

In [5]:
data.fillna(0,inplace=True)

In [6]:
df = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Переведём категориальные признаки в численный вид.

In [8]:
df.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

Возпользуемся методом `get_dummies`.

In [9]:
df = pd.get_dummies(df, drop_first=True)

In [10]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Данные готовы к исследованию на разброс и баланс классов.

In [11]:
df.shape

(10000, 12)

## Исследование задачи

In [12]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Exited имеет неравные классы, обучим модель без учета этого

In [13]:
features = df.drop('Exited', axis=1)
target =  df['Exited']

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size = 0.40, random_state = 12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size = 0.40, random_state = 12345)

### Логистическая регрессия
Обучим линейную модель. Проверим её качество.

In [15]:
%%time

lin_model = LogisticRegression(random_state=12345, solver='liblinear')
lin_model.fit(features_train, target_train)

Wall time: 48 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [16]:
f1_score(target_valid, lin_model.predict(features_valid)).round(3)

0.095

In [17]:
roc_auc_score(target_valid, lin_model.predict_proba(features_valid)[:,1]).round(3)

0.659

Значение f1  = 0.095, что говорит о низком качестве модели. По AUC-ROC можно судить, что модель предсказывает лучше случайной, возможно сказывается несбалансированность классов.

### Решающие деревья
Обучим решающее дерево. Подберем наилучшую глубину дерева. Проверим качество моедли метриками F1 и auc-roc .

In [18]:
%%time

depth = 0
tree_model = None
f1 = 0

for dep in range(2,16):
    model = DecisionTreeClassifier(max_depth=dep,random_state=12345)
    model.fit(features_train,target_train)
    F1 = f1_score(target_valid,model.predict(features_valid))
    if F1 > f1:
        f1 = F1
        tree_model = model
        depth = dep

Wall time: 583 ms


In [19]:
print('Значение f1: {:.2f} \nГлубина дерева: {} \nAUC-ROC: {:.2f}'.format(
    f1,
    depth,
    roc_auc_score(target_valid,tree_model.predict_proba(features_valid)[:,1]))
     )

Значение f1: 0.57 
Глубина дерева: 9 
AUC-ROC: 0.80


Качество модели заметно выросло, но всё ещё не дотягивает до порогового (f1 > 0.57). Модель работает быстро, менее чем за секунду.

### Случайный лес
Обучим модель случайного леса, подобрав оптимальные гиперпараметры.  

<b> Оптимальная глубина</b>

In [20]:
%%time

forest_depth = 0
forest_model = None
f1 = 0
for dep in range(1, 16):
    model = RandomForestClassifier(n_estimators=20, max_depth=dep, random_state=12345)
    model.fit(features_train, target_train)
    F1 = f1_score(target_valid, model.predict(features_valid))
    if F1 > f1:
        forest_model = model
        f1 = F1
        forest_depth = dep

Wall time: 2.75 s


In [21]:
print('Значение f1: {:.2f} \nГлубина деревьев: {} \nAUC-ROC: {:.2f}'.format(
    f1,
    forest_depth,
    roc_auc_score(target_valid,forest_model.predict_proba(features_valid)[:,1]))
     )

Значение f1: 0.58 
Глубина деревьев: 12 
AUC-ROC: 0.84


<b> Оптимальное количество деревьев</b>

In [22]:
%%time

count = 20
for N in range(25, 40, 1):
    model = RandomForestClassifier(n_estimators=N, max_depth=forest_depth, random_state=12345)
    model.fit(features_train, target_train)
    F1 = f1_score(target_valid, model.predict(features_valid))
    if F1 > f1:
        forest_model = model
        f1 = F1
        count = N

Wall time: 6.32 s


In [23]:
    print('Значение f1: {:.2f} \nКоличество деревьев: {} \nAUC-ROC: {:.2f}'.format(
    f1,
    count,
    roc_auc_score(target_valid,forest_model.predict_proba(features_valid)[:,1]))
     )

Значение f1: 0.58 
Количество деревьев: 37 
AUC-ROC: 0.84


Увеличение количества деревьев не сильно повлияло на качество модели. Получилось добиться значения f1 = 0.58. AUC-ROC равно 0.84, это говорит нам о том, что модель намного лучше случайной, но до идеальной ещё далеко.

## Борьба с дисбалансом
Попробуем улучшить качество модели сбалансировав классы.

Сбалансируем классы техникой upsampling.

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [25]:
features_train, target_train = upsample(features_train, target_train, 4)

### Логистическая регрессия
Обучим линейную модель. Проверим разные проговые значения для определения класса.

In [26]:
lin_model_balance = LogisticRegression(random_state=12345, solver='liblinear')
lin_model_balance.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [27]:
probabilities_valid = lin_model_balance.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.4, 0.6, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    
    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        threshold, precision, recall))
    print('Значение f1: {:.2f}'.format(
    f1_score(target_valid, predicted_valid)))

Порог = 0.40 | Точность = 0.271, Полнота = 0.842
Значение f1: 0.41
Порог = 0.42 | Точность = 0.282, Полнота = 0.815
Значение f1: 0.42
Порог = 0.44 | Точность = 0.295, Полнота = 0.775
Значение f1: 0.43
Порог = 0.46 | Точность = 0.304, Полнота = 0.736
Значение f1: 0.43
Порог = 0.48 | Точность = 0.313, Полнота = 0.696
Значение f1: 0.43
Порог = 0.50 | Точность = 0.332, Полнота = 0.661
Значение f1: 0.44
Порог = 0.52 | Точность = 0.344, Полнота = 0.621
Значение f1: 0.44
Порог = 0.54 | Точность = 0.363, Полнота = 0.586
Значение f1: 0.45
Порог = 0.56 | Точность = 0.374, Полнота = 0.544
Значение f1: 0.44
Порог = 0.58 | Точность = 0.376, Полнота = 0.487
Значение f1: 0.42


In [28]:
f1_score(target_valid, lin_model_balance.predict(features_valid)).round(3)

0.442

In [29]:
roc_auc_score(target_valid, lin_model_balance.predict_proba(features_valid)[:,1]).round(3)

0.708

Качество модели заметно выросло с f1 до 0.45, значение порога оптимально на 0.54.

### Решающие деревья
Обучим решающее дерево. Подберем наилучшую глубину дерева. Проверим качество моедли метриками F1 и auc-roc .

In [30]:
tree_model_balance = DecisionTreeClassifier(max_depth=5,random_state=12345)
tree_model_balance.fit(features_train,target_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=12345, splitter='best')

In [31]:
print('Значение f1: {:.2f} \nAUC-ROC: {:.2f}'.format(
    f1_score(target_valid, tree_model_balance.predict(features_valid)),
    roc_auc_score(target_valid,tree_model_balance.predict_proba(features_valid)[:,1]))
     )

Значение f1: 0.59 
AUC-ROC: 0.83


Для решающего дерева качество увеличилось до минимально необходимого.

### Случайный лес
Обучим модель случайного леса, подобрав оптимальные гиперпараметры.  

In [32]:
forest_model_balance = RandomForestClassifier(
    n_estimators=count, max_depth=forest_depth, random_state=12345
)
forest_model_balance.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=37,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [33]:
print('Значение f1: {:.2f} \nAUC-ROC: {:.2f}'.format(
    f1_score(target_valid, forest_model_balance.predict(features_valid)),
    roc_auc_score(target_valid,forest_model_balance.predict_proba(features_valid)[:,1]))
     )

Значение f1: 0.61 
AUC-ROC: 0.84


Для случайного леса качество возросло до необходимого.

## Тестирование модели
Наилучший и приемлимый результат выдает только модель случайного леса с глубиной 12 и количеством деревьев 21.

In [34]:
forest_model_balance = RandomForestClassifier(
    n_estimators=count, max_depth=forest_depth, random_state=12345
)
forest_model_balance.fit(features_train, target_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=37,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [35]:
print('Значение f1 для валидационной выборки: {:.2f} \nДля тестовой: {:.2f}'.format(
    f1_score(
        target_valid, forest_model_balance.predict(features_valid)),
    f1_score(
        target_test, forest_model_balance.predict(features_test))))

print('Значение AUC-ROC для валидационной выборки: {:.2f} \nДля тестовой: {:.2f}'.format(
    roc_auc_score(
        target_valid,forest_model_balance.predict_proba(features_valid)[:,1]),
    roc_auc_score(
        target_test,forest_model_balance.predict_proba(features_test)[:,1])
     ))


Значение f1 для валидационной выборки: 0.61 
Для тестовой: 0.62
Значение AUC-ROC для валидационной выборки: 0.84 
Для тестовой: 0.85


In [36]:
print('Точность предсказания для валидационной выборки: {:.2%} \nДля тестовой: {:.2%}'.format(
    precision_score(target_valid,forest_model_balance.predict(features_valid)),
    precision_score(
        target_test,forest_model_balance.predict(features_test))))

Точность предсказания для валидационной выборки: 58.93% 
Для тестовой: 61.30%


## Вывод 
Лучшей моделью является модель случайного леса, она предсказывает с точностью 59-60%. Значение метрики f1 составило 61-62%.

Также хорошо себя показала модель решающего дерева, но при балансировки классов не показала значимого улучшения качества.

<a id="Вывод"></a>